In [57]:
!pip install -q -U git+https://github.com/huggingface/transformers.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [58]:
pip install ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [59]:
pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [60]:
pip install langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [61]:
pip install pytesseract

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [62]:
pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [63]:
# import all the necessary modules
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFaceHub
import torch.nn.functional
from PIL import Image
from langchain.chat_models import ChatOpenAI
import pytesseract
import ollama
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer
import requests
from PIL import Image
import torch
import pandas as pd
import time
from datetime import datetime

In [64]:
# use google deplot hungging face api to generate the underlying data table for the chart passed in
def ExtractChartMetadata(image_file):
  device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
  processor = Pix2StructProcessor.from_pretrained('google/deplot')
  model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot').to(device)

  model.to(device)
  image = Image.open(image_file)
  inputs = processor(images = image, text = 'Generate underlying data table of the figure below:',return_tensors='pt').to(device)
  inputs = {k: v.to(device) for k,v in inputs.items()}
  predictions = model.generate(**inputs,max_new_tokens = 512)

  output_text = processor.decode(predictions[0].to('cpu'),skip_special_tokens=True)
  torch.mps.empty_cache()
  return output_text


In [65]:
def ParseRawTextDF(raw_text):
    # Replace <0x0A> with a newline character for easier processing
    raw_text = raw_text.replace("<0x0A>", "\n")
    # Split the text into lines
    lines = raw_text.split("\n")
    # Dynamically parse each line into a list of columns
    data = [line.split(" | ") for line in lines if line]  # Ensure no empty lines are processed

    # Handle variable number of columns by checking the length of each row
    max_cols = max(len(row) for row in data)
    columns = data[0] if len(data[0]) == max_cols else [f"Column {i+1}" for i in range(max_cols)]

    # Ensure all rows have the same number of columns, filling missing values with None
    data_normalized = [row + [None] * (max_cols - len(row)) for row in data]

    # Convert to DataFrame using the normalized data
    df = pd.DataFrame(data_normalized[1:], columns=columns)
    return df

In [66]:
import ollama
from PIL import Image
import base64
import numpy as np

In [67]:
# load chart dataset that will be used for inference 
from datasets import load_dataset

dataset = load_dataset("khhuang/CHOCOLATE",split = 'test')

In [68]:
def OllamaPullLLM(llm_choice):
    if llm_choice == "gemma":
      ollama.pull('gemma:7b')
      model_name = 'gemma:7b'
      
    elif llm_choice == "llama":
      ollama.pull('llama2')
      model_name = 'llama2'
      
    elif llm_choice == 'mistral':
      ollama.pull('mistral')
      model_name = 'mistral'
      
    elif llm_choice == 'codellama':
      ollama.pull('codellama')
      model_name = 'codellama'
      
    elif llm_choice == 'llava':
      ollama.pull('llava')
      model_name = 'llava'
      
    else:
      ollama.pull('llama2')
      model_name = 'llama2'

    return model_name
    #tokenizer = AutoTokenizer.from_pretrained(llm_name)
    #model = AutoModelForCausalLM.from_pretrained(llm_name)
    #return tokenizer, model

In [69]:
def generate_chart_description(metadata,llm_choice):
  tokenizer,model = prepare_llm(llm_choice)

  device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
  model.to(device)

  prompt = f"Based on the following chart data: {metadata}, describe the chart's key features and trends."

  inputs = tokenizer(prompt,return_tensors='pt')
  inputs.to(device)
  outputs = model.generate(**inputs,max_new_tokens = 200, num_return_sequences=1)
  outputs.to(device)
  description = tokenizer.decode(outputs[0],skip_special_tokens = True)

  return description

In [70]:
def GenerateChartDescription_Ollama(metadata,image_path,llm_choice):
    
    llm = OllamaPullLLM(llm_choice)
    
    image = Image.open(image_path)
    image = image.resize((224, 224))  # Resize to match model input size
    image = np.array(image) / 255.0  # Normalize pixel values
        
    stream = ollama.chat(model = llm, messages =  [{'role': 'user', 'content': f"You are a part of a chart-to-text deep learning pipeline. The first step of the pipeline involved taking an image of a chart and extracting metadata in tabular format. The next step of the pipeline is for you to Describe what is happening in the following chart image {image}. Use the following extracted chart metadata {metadata} Please describe what is happening in this chart image you explain what is happening in this chart image {image} with the following extracted chart metadata in tabular format as a guide {metadata}?"}])    
    
    return stream

In [71]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image

def PrepareScoringModel():
    model_name = 'khhuang/chartve'
    model = VisionEncoderDecoderModel.from_pretrained(model_name)
    processor = DonutProcessor.from_pretrained(model_name)
    return model,processor

def CaptionAccuracyScoring(model,processor,sentence,image_path):
    
    def format_query(sentence_string):
        return f"Does the image entail this statement: \"{sentence_string}\"?"

    # Format text inputs
    query = format_query(sentence)

    # Encode chart figure and tokenize text
    img = Image.open(image_path)
    pixel_values = processor(img.convert("RGB"), random_padding=False, return_tensors="pt").pixel_values
    decoder_input_ids = processor.tokenizer(query, add_special_tokens=False, return_tensors="pt", max_length=510).input_ids


    outputs = model(pixel_values, decoder_input_ids=decoder_input_ids)

    # Probe the probability of generating "yes"
    binary_entail_prob_positive = torch.nn.functional.softmax(outputs['logits'].squeeze()[-1,[2334, 49922]])[1].item()

    # binary_entail_prob_positive corresponds to the computed probability that the chart entails the caption sentence.
    return binary_entail_prob_positive

In [73]:
def ScoreChartDescription(chatstream,image_path,llm_choice,metadata):
    description = chatstream['message']['content']
    sentence_segmented_descr = description.split('.')
    
    print("Here is the chart description generated by the first pass: ")
    print(description)
    
    print("Preparing Scoring Model...")
    model, processor = PrepareScoringModel()
    print("Finished Preparing Scoring Model")
    
    allSentencesAccurateFlag = True
    AccuracyStr = {'caption_sentences':[], 'accuracy_score' : []}
    for sentence in sentence_segmented_descr:
        accuracy_score = CaptionAccuracyScoring(model, processor, sentence,image_path)
        AccuracyStr['caption_sentences'].append(sentence)
        AccuracyStr['accuracy_score'].append(accuracy_score)
        if accuracy_score < 0.93:
            allSentencesAccurateFlag = False

    print("Scored accuracies for caption sentences:")
    print(AccuracyStr)

    if not allSentencesAccurateFlag:
        # make a generic function for use with ollama.chat that takes as input the prompt we want to use
        # we have rewritten a lot of code below
        
        llm = OllamaPullLLM(llm_choice)
        image = Image.open(image_path)
        image = image.resize((224, 224))  # Resize to match model input size
        image = np.array(image) / 255.0  # Normalize pixel values
        
        for idx, sentence in enumerate(AccuracyStr['caption_sentences']):
            best_scoring_candidate_replacement = sentence
            best_score_replacement = AccuracyStr['accuracy_score'][idx]
            
            sentence_so_far = sentence
            i = 0
            while (best_score_replacement < 0.93) and (i < 5):
                stream = ollama.chat(model = llm, messages = [{'role': 'user', 'content': f"The following is an image of a chart {image}. Using this chart image and the following extracted chart metadata in tabular format as a guide {metadata}, you generated the following description {description}. This sentence, however, was not very accurate: {sentence}. Generate a single correct sentence to replace the incorrect sentence with a corrected version using the chart image, the metadata, and the description provided to you. Do not preface your answer with a reply simply output the corrected sentence."}])
                sentence_so_far = stream['message']['content']
                i += 1
                accuracy_score_sentence_so_far = CaptionAccuracyScoring(model, processor, sentence_so_far,image_path)
                if accuracy_score_sentence_so_far > best_score_replacement:
                    best_score_replacement = accuracy_score_sentence_so_far
                    best_scoring_candidate_replacement = sentence_so_far
                    
            AccuracyStr['caption_sentences'][idx] = best_scoring_candidate_replacement
            AccuracyStr['accuracy_score'][idx] = best_score_replacement
            
    return AccuracyStr
        

In [74]:
'''
i=1

for sample in dataset:
  image_path = sample['image_path']  # Adjust this key based on the actual dataset structure
  save_path = 'TheBoard/ChartImages/Chart_Example_'+str(i)+'.png'
  i+=1
  torch.hub.download_url_to_file(image_path, save_path)
'''

"\ni=1\n\nfor sample in dataset:\n  image_path = sample['image_path']  # Adjust this key based on the actual dataset structure\n  save_path = 'TheBoard/ChartImages/Chart_Example_'+str(i)+'.png'\n  i+=1\n  torch.hub.download_url_to_file(image_path, save_path)\n"

In [75]:
image_path = 'TheBoard/ChartImages/Chart_Example_8.png'

In [76]:
# it took 10 minutes almost to load deplot model weights but under a minute to make the prediction
metadata = ExtractChartMetadata(image_path)

In [77]:
metadata

'TITLE | Number of racially motivated hate crimes in England and Wales from April 2019 to March 2020, by region<0x0A>region | Number of hate crimes*<0x0A>in England | Wales from April 2019 to March 2020<0x0A>in England and Wales from April 2019 to March 2020<0x0A>2020 | <0x0A> West Midlands | 9.4 | 8.1 | 8.1 | 7.6 <0x0A> Yorkshire and the Humber | 9.4 | 7.8 | 4.8 | 4.6 <0x0A> South West | 10.5 | 10.5 | 5.4 | 10.5 <0x0A> North West | 5.4 | 7.2 | 3.1 | 3.1 <0x0A> London | 17.6 | 17.7 | 17.5 | 17.5 <0x0A> East Midlands | 4.8 | 4.6 | 4.6 | 4.8'

In [78]:

stream = GenerateChartDescription_Ollama(metadata = metadata,image_path = image_path,llm_choice = 'llama')

In [79]:
stream

{'model': 'llama2',
 'created_at': '2024-03-21T00:10:30.173456Z',
 'message': {'role': 'assistant',
  'content': 'The chart image you provided shows the number of racially motivated hate crimes in England and Wales from April 2019 to March 2020, broken down by region. The metadata extracted from the chart is as follows:\n\nTITLE | Number of racially motivated hate crimes in England and Wales from April 2019 to March 2020, by region\n\nregion | Number of hate crimes*\n\nin England | Wales from April 2019 to March 2020\n\nin England and Wales from April 2019 to March 2020\n\n2020 |\n\nWest Midlands | 9.4 | 8.1 | 8.1 | 7.6\n\nYorkshire and the Humber | 9.4 | 7.8 | 4.8 | 4.6\n\nSouth West | 10.5 | 10.5 | 5.4 | 10.5\n\nNorth West | 5.4 | 7.2 | 3.1 | 3.1\n\nLondon | 17.6 | 17.7 | 17.5 | 17.5\n\nEast Midlands | 4.8 | 4.6 | 4.6 | 4.8\n\n*Note: The numbers are represented as bars in the chart, with the height of each bar indicating the number of hate crimes in that region.'},
 'done': True,
 't

In [80]:
import warnings
warnings.filterwarnings('ignore')
AccuracyStr = ScoreChartDescription(chatstream = stream,image_path = image_path,llm_choice = 'llama',metadata = metadata)

Here is the chart description generated by the first pass: 
The chart image you provided shows the number of racially motivated hate crimes in England and Wales from April 2019 to March 2020, broken down by region. The metadata extracted from the chart is as follows:

TITLE | Number of racially motivated hate crimes in England and Wales from April 2019 to March 2020, by region

region | Number of hate crimes*

in England | Wales from April 2019 to March 2020

in England and Wales from April 2019 to March 2020

2020 |

West Midlands | 9.4 | 8.1 | 8.1 | 7.6

Yorkshire and the Humber | 9.4 | 7.8 | 4.8 | 4.6

South West | 10.5 | 10.5 | 5.4 | 10.5

North West | 5.4 | 7.2 | 3.1 | 3.1

London | 17.6 | 17.7 | 17.5 | 17.5

East Midlands | 4.8 | 4.6 | 4.6 | 4.8

*Note: The numbers are represented as bars in the chart, with the height of each bar indicating the number of hate crimes in that region.
Preparing Scoring Model...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Finished Preparing Scoring Model
Scored accuracies for caption sentences:
{'caption_sentences': ['The chart image you provided shows the number of racially motivated hate crimes in England and Wales from April 2019 to March 2020, broken down by region', ' The metadata extracted from the chart is as follows:\n\nTITLE | Number of racially motivated hate crimes in England and Wales from April 2019 to March 2020, by region\n\nregion | Number of hate crimes*\n\nin England | Wales from April 2019 to March 2020\n\nin England and Wales from April 2019 to March 2020\n\n2020 |\n\nWest Midlands | 9', '4 | 8', '1 | 8', '1 | 7', '6\n\nYorkshire and the Humber | 9', '4 | 7', '8 | 4', '8 | 4', '6\n\nSouth West | 10', '5 | 10', '5 | 5', '4 | 10', '5\n\nNorth West | 5', '4 | 7', '2 | 3', '1 | 3', '1\n\nLondon | 17', '6 | 17', '7 | 17', '5 | 17', '5\n\nEast Midlands | 4', '8 | 4', '6 | 4', '6 | 4', '8\n\n*Note: The numbers are represented as bars in the chart, with the height of each bar indicating the 

In [81]:
AccuracyStr

{'caption_sentences': ['The chart image you provided shows the number of racially motivated hate crimes in England and Wales from April 2019 to March 2020, broken down by region',
  'The number of racially motivated hate crimes in England and Wales from April 2019 to March 2020 was highest in the West Midlands region, with 9.4 incidents in total, followed by the Yorkshire and the Humber region with 9.4 incidents.',
  'The number of racially motivated hate crimes in England and Wales from April 2019 to March 2020 was highest in London, with 17.6 hate crimes reported in that region alone.',
  'The number of racially motivated hate crimes in England and Wales from April 2019 to March 2020 was highest in the West Midlands with 9.4 incidents, followed by Yorkshire and the Humber with 9.4 incidents.',
  'The number of racially motivated hate crimes in England and Wales from April 2019 to March 2020 was highest in London, with 17.6 hate crimes in that region alone.',
  'The number of racially

In [ ]:
image_path = 'TheBoard/ChartImages/Chart_Example_9.png'
image = Image.open(image_path)
image = image.resize((224, 224))  # Resize to match model input size
image = np.array(image) / 255.0  # Normalize pixel values

In [56]:
metadata

'TITLE | Number of social network users worldwide in 2020, by region (in millions)<0x0A>geographic region | Number of active social media users in millions <0x0A> Western Asia | 164.82 <0x0A> Western Africa | 38.22 <0x0A> Southern Europe | 151.22 <0x0A> Southern Africa | 17.15 <0x0A> Southeast Asia | 479.15 <0x0A> South America | 260.0 <0x0A> Northern America | 326.93 <0x0A> Northern Africa | 106.51 <0x0A> Eastern Asia | 102.97 <0x0A> Eastern Africa | 30.53 <0x0A> Central Asia | 10.92 <0x0A> Central Africa | 11.17 <0x0A> Caribbean | 10.35 <0x0A> Australia & Oceania | 10.3'

In [51]:
streamLLAMA = ollama.chat(model = 'llama2', messages =  [{'role': 'user', 'content': f"You are a part of a chart-to-text deep learning pipeline. The first step of the pipeline involved taking an image of a chart and extracting metadata in tabular format. The next step of the pipeline is for you to take the extracted metadata and the image of the chart and use this supplied information to describe what is happening in the chart image. I want you to pay special attention to the title, any legends, x-axis, and y-axis labels, and what the data points represent. Here is the metadata: {metadata}. Here is the image: {image}."}])    


In [53]:
streamLLAVA = ollama.chat(model = 'llava', messages =  [{'role': 'user', 'content': f"You are a part of a chart-to-text deep learning pipeline. The first step of the pipeline involved taking an image of a chart and extracting metadata in tabular format. The next step of the pipeline is for you to take the extracted metadata and the image of the chart and use this supplied information to describe what is happening in the chart image. I want you to pay special attention to the title, any legends, x-axis, and y-axis labels, and what the data points represent. Here is the metadata: {metadata}. Here is the image: {image}."}])    


In [52]:
streamLLAMA

{'model': 'llama2',
 'created_at': '2024-03-17T19:07:27.086878Z',
 'message': {'role': 'assistant',
  'content': 'Based on the provided image and metadata, I can identify the following:\n\nTitle: The chart shows the number of social network users worldwide in 2020, broken down by region.\n\nGeographic Region: The chart provides the number of active social media users in millions for each of the following regions:\n\n* Western Asia: 164.82 million\n* Western Africa: 38.22 million\n* Southern Europe: 151.22 million\n* Southern Africa: 17.15 million\n* Southeast Asia: 479.15 million\n* South America: 260.0 million\n* Northern America: 326.93 million\n* Northern Africa: 106.51 million\n* Eastern Asia: 102.97 million\n* Eastern Africa: 30.53 million\n* Central Asia: 10.92 million\n* Central Africa: 11.17 million\n* Caribbean: 10.35 million\n* Australia & Oceania: 10.3 million\n\nThe chart shows a total of 1,486.73 million active social media users worldwide in 2020, with the majority residi

In [55]:
streamLLAVA

{'model': 'llava',
 'created_at': '2024-03-17T19:07:32.824896Z',
 'message': {'role': 'assistant',
  'content': ' The image you have provided appears to be a blank page with no content visible. Please check the URL or file path of the image and ensure that it is correct before providing it again for further processing in the chart-to-text pipeline. If you continue to experience issues, please let me know so I can assist you further. '},
 'done': True,
 'total_duration': 5693369333,
 'load_duration': 635018666,
 'prompt_eval_count': 1788,
 'prompt_eval_duration': 3466112000,
 'eval_count': 68,
 'eval_duration': 1590360000}

In [ ]:
torch.mps.empty_cache()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()